In [ ]:
# 알라딘 사이트에서 원하는 기간의 베스트 셀러를 딕셔너리 행태로 반환해주는 함수
def aladin_BestSeller(year, month, week, rank):
  # year : 해당 년도의 베스트 셀러
  # month : 해당 달의 베스트 셀러
  # week : 해당 주의 베스트 셀러
  # rank : 순위 설정 (최대 20 -> 1000위까지)
  # 1 : 1 ~ 50위 / 2 : 1 ~ 100위 / 3 : 1 ~ 150위 ...


  # 필요한 라이브러리 불러오기
  from tqdm.notebook import tqdm
  import requests
  from bs4 import BeautifulSoup

  # 해당 년, 달, 주, 순위의 베스트셀러 정보를 담는 딕셔너리
  dict_best_seller_info = {
      '순위':[], # 해당 기간 베스트 셀러 순위
      '출시':[], # 베스트셀러 출시 날짜
      '제목':[], # 베스트셀러 제목
      '지은이':[], # 베스트셀러 작가
      '출판사':[], # 베스트셀러 출판사
      '쪽':[], # 베스트셀러 쪽
      '분류':[], # 베스트셀러 카테고리
      '정가':[], # 베스트셀러 정가
      '베스트셀러 기간':[], # 설정한 베스트 셀러 기간 (년도-달-주)
      '중고 판매 예상가':[], # 알라딘 팔기 예상가 (최상)
      '중고 재고':[], # 중고 서적 재고 량
      '평점':[], # 10점 만점
      'Sales Point':[] # 판매량과 판매기간에 근거하여 해당 상품의 판매도를 산출한 알라딘만의 판매지수법
  }

  list_best_seller_title = [] # 베스트 셀러 제목 리스트
  list_best_seller_ID = [] # 베스트 셀러 ID 리스트 
  dict_best_seller_title_ID_rank = {} # 베스트 셀러 제목에 상응하는 ID 담는 딕셔너리

  for ranking in range(1, rank+1):
    url_aladin = 'https://www.aladin.co.kr/shop/common/wbest.aspx?'
    params = {
        'BestType':'Bestseller',
        'BranchType':1,
        'CID':0,
        'Year':year,
        'Month':month,
        'Week':week,
        'page':ranking,
        'cnt':1000,
        'SortOrder':1
    }


    # 원하는 기간의 알라딘 베스트 셀러 사이트 접속
    resp_aladin = requests.get(url_aladin, params=params)

    soup_aladin = BeautifulSoup(resp_aladin.content, 'lxml')
    
    # 베스트 셀러 제목과 ID 
    best_seller_tags = soup_aladin.select('div.ss_book_list ul li a.bo3')
    
    # 베스트 셀러의 제목과 ID를 각각의 리스트에 담기
    for best_seller_tag in best_seller_tags:
      list_best_seller_title.append(best_seller_tag.text)
      list_best_seller_ID.append(best_seller_tag['href'].split('=')[-1])

    
  for title, ID in zip(list_best_seller_title, list_best_seller_ID):
      dict_best_seller_title_ID_rank[title] = ID
  
  rank_num = 1 # 기본 순위는 1부터 시작한다

  for book_title in tqdm(list(dict_best_seller_title_ID_rank.keys()), desc='{}년 {}월 {}주 베스트 셀러'.format(year, month, week)):
    book_url = 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId='+dict_best_seller_title_ID_rank[book_title]
    
    resp_book = requests.get(book_url)

    soup_book = BeautifulSoup(resp_book.content, 'lxml')

    try:

      # 출시
      publish_tags = soup_book.select('li.Ere_sub2_title')
      if publish_tags != []:
        for publish_tag in publish_tags:
          if '원제' in publish_tag.text:
            dict_best_seller_info['출시'].append(publish_tag.text.split('원제')[-2][-10:])
          else:
            dict_best_seller_info['출시'].append(publish_tag.text[-10:])
      else:
        dict_best_seller_info['출시'].append(None)

      # 순위
      if publish_tags != []: # 제목별 사이트 사이트에 접속하게 되면
        # ranking에 상응하는 순위 알파 값을 준다
        dict_best_seller_info['순위'].append(rank_num)
      else:
        dict_best_seller_info['순위'].append(None)

      # 베스트 셀러 기간
      if publish_tags != []:
        dict_best_seller_info['베스트셀러 기간'].append(str(year)+'-'+str(month)+'-'+str(week))
      else:
        dict_best_seller_info['베스트셀러 기간'].append(None)

      # 중고 팔기 예상가
      try:
        if publish_tags != []:
          # 중고 사이트에 접속
          url_used = 'https://www.aladin.co.kr/shop/UsedShop/wuseditemall.aspx?ItemId=' + dict_best_seller_title_ID_rank[book_title]
          
          resp_used = requests.get(url_used)
          
          soup_used = BeautifulSoup(resp_used.content, 'lxml')
          # 알라딘 팔기 예상가에서 최상인 경우
          used_tags = soup_used.select('div.Ere_usednum_box table tr td.Tit_tableCF')[0]

          # '알라딘 중고 팔기 최상가'가 없는 경우 Null값을 준다
          if used_tags.text == '-':
            dict_best_seller_info['중고 판매 예상가'].append(None)
          else:
            dict_best_seller_info['중고 판매 예상가'].append(int(used_tags.text.strip('원').replace(',', '')))
        else:
          dict_best_seller_info['중고 판매 예상가'].append(None)
      except:
        dict_best_seller_info['중고 판매 예상가'].append(None)

      # 중고 재고
      try:
        if publish_tags != []:
          stock_tags = soup_used.select('a.Ere_usedsell_tab_on')[0]
          
          dict_best_seller_info['중고 재고'].append(int(stock_tags.text.split(' ')[-1].strip('(').strip(')')))
        else:
          dict_best_seller_info['중고 재고'].append(None)

      except:
        dict_best_seller_info['중고 재고'].append(None)  


      # 제목
      try:
        title_tags = soup_book.select('a.Ere_bo_title')
        if title_tags != []:
          for title_tag in title_tags:
            dict_best_seller_info['제목'].append(title_tag.text)
        else:
          dict_best_seller_info['제목'].append(book_title)
      except:
        dict_best_seller_info['제목'].append(book_title)

      # 지은이
      try:
        author_tags = soup_book.select('a.Ere_sub2_title')[0]
        if len(author_tags.text) == 0:
            dict_best_seller_info['지은이'].append(None)
        else:
            for author_tag in author_tags:
              dict_best_seller_info['지은이'].append(author_tag)
      except:
        dict_best_seller_info['지은이'].append(None)

      # 책 출판사 : 원제가 있는 경우 태그를 다르게 한다
      try:
        publisher_tags = soup_book.select('a.Ere_sub2_title')[-1]

        if '원제' in publisher_tags.text:
          publisher_tags = soup_book.select('a.Ere_sub2_title')[-2]
          for publisher_tag in publisher_tags:
            dict_best_seller_info['출판사'].append(publisher_tag)
        
        else:
          for publisher_tag in publisher_tags:
            dict_best_seller_info['출판사'].append(publisher_tag)
      except:
        dict_best_seller_info['출판사'].append(None)

      # 책 쪽 : 양장본 및 기타가 있는 경우 태그값을 다르게 함
      try:
        page_tags = soup_book.select('div.conts_info_list1 ul li')[0]

        for page_tag in page_tags:
          if page_tag == '양장본':
            page_tags = soup_book.select('div.conts_info_list1 ul li')[1]
            dict_best_seller_info['쪽'].append(int(page_tags.text.strip('쪽')))
          elif page_tag == '기타':
            page_tags = soup_book.select('div.conts_info_list1 ul li')[1]
            dict_best_seller_info['쪽'].append(int(page_tags.text.strip('쪽')))
          else:
            dict_best_seller_info['쪽'].append(int(page_tags.text.strip('쪽')))
      except:
        dict_best_seller_info['쪽'].append(None)

      # 책 주제 분류 : 두번째 기준 / 분류가 없는 경우 Null
      try:
        category_tags = soup_book.select('ul#ulCategory li a')

        if category_tags == []:
          dict_best_seller_info['분류'].append(None)
        else:
          for category_tag in category_tags[1]:
            dict_best_seller_info['분류'].append(category_tag)
      except:
        dict_best_seller_info['분류'].append(None)

      # 책 정가 / 정가를 할인하는 경우 태그를 다르게 함
      try:
        price_tags = soup_book.select('div.Ritem')[0]

        try:
          for price_tag in price_tags:
            dict_best_seller_info['정가'].append((int(price_tag.strip(' ').strip('원').replace(',', ''))))
        
        except:
          price_tags = soup_book.select('div.info_list ul li div.Ritem del')[0]

          for price_tag in price_tags:
            dict_best_seller_info['정가'].append((int(price_tag.strip(' ').strip('원').replace(',', ''))))
      except:
        dict_best_seller_info['정가'].append(None)

      # 책 평점 : 10점 만점에서의 평점
      try:
        star_tags = soup_book.select('div.info_list a.Ere_sub_pink')
        if star_tags != []:
          for star_tag in star_tags:
            dict_best_seller_info['평점'].append(float(star_tag.text.replace(',', '')))
        else:
          dict_best_seller_info['평점'].append(None)
      except:
        dict_best_seller_info['평점'].append(None)

      # 책 Sales Point
      try:
        sales_point_tags = soup_book.select('div.Ere_fs15 strong')[-1]

        for sales_point_tag in sales_point_tags:
          dict_best_seller_info['Sales Point'].append(int(sales_point_tag.replace(',', '')))
      except:
         dict_best_seller_info['Sales Point'].append(None)

      rank_num += 1

    except:
      print('도서명 : "{}"는 알라딘 베스트셀러 크롤링에서 제외시킵니다.'.format(book_title))
      rank_num += 1
      pass

  return dict_best_seller_info

In [ ]:
test = aladin_BestSeller(2021, 12, 5, 1)


2021년 12월 5주 베스트 셀러:   0%|          | 0/50 [00:00<?, ?it/s]

AttributeError: ignored

In [ ]:
import pandas as pd

test = pd.DataFrame(test)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   순위           49 non-null     float64
 1   출시           49 non-null     object 
 2   제목           50 non-null     object 
 3   지은이          49 non-null     object 
 4   출판사          49 non-null     object 
 5   쪽            49 non-null     float64
 6   분류           49 non-null     object 
 7   정가           49 non-null     float64
 8   베스트셀러 기간     49 non-null     object 
 9   중고 판매 예상가    44 non-null     float64
 10  중고 재고        49 non-null     float64
 11  평점           49 non-null     float64
 12  Sales Point  49 non-null     float64
dtypes: float64(7), object(6)
memory usage: 5.2+ KB
